In [20]:
import tensorflow.keras as keras
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.utils import shuffle, class_weight
import pandas as pd
import numpy as np
import tensorflow as tf

In [21]:
path = r"Overall_DA_Dataset_with_APR.csv"
df = pd.read_csv(path)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78712 entries, 0 to 78711
Columns: 105 entries, x_mean to Labels
dtypes: float64(86), int64(18), object(1)
memory usage: 63.1+ MB


In [23]:
df.Labels.value_counts()

Asphalt_Ride        51190
Stop                14193
Cobblestone_Ride     9508
Asphalt_Manhole      2824
Asphalt_Kurb          621
Asphalt_Bump          376
Name: Labels, dtype: int64

In [24]:
class_weight = {0: 1.,
                1: 3.,
                2: 5.,
               3: 22.,
               4: 80.,
               5: 130.,
               }

In [25]:
N_train_samples = 70000
N_test_samples = 8712
N_samples = N_train_samples + N_test_samples
N_epochs = 100
batch_size = 64
learning_rate = 0.01

In [26]:
X = df.drop('Labels', axis=1)
y = df["Labels"]

In [27]:
y = np.expand_dims(y, axis=-1)

# encode class values as integers
encoder = OneHotEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y = encoded_y.todense()

In [28]:
shuffled_x, shuffled_y= shuffle(X, encoded_y)

In [29]:
x_train = shuffled_x[0:N_train_samples]
y_train = shuffled_y[0:N_train_samples]

x_test = shuffled_x[N_train_samples: N_train_samples+N_test_samples]
y_test = shuffled_y[N_train_samples: N_train_samples+N_test_samples]

In [30]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size, drop_remainder=True).shuffle(1000).repeat()

In [31]:
#x_train, x_test, y_train, y_test = train_test_split(X.values, encoded_y, test_size=0.1)

In [32]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(101, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(6, activation='softmax'))

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_ds,  batch_size=batch_size, verbose=1, epochs=N_epochs, steps_per_epoch = N_samples/batch_size, class_weight = class_weight)

Epoch 1/100
1229/1229 [==============================] - 7s 4ms/step - loss: 25.8852 - accuracy: 0.7397
Epoch 2/100
1229/1229 [==============================] - 5s 4ms/step - loss: 11.5904 - accuracy: 0.8370
Epoch 3/100
1229/1229 [==============================] - 5s 4ms/step - loss: 8.9809 - accuracy: 0.8646
Epoch 4/100
1229/1229 [==============================] - 6s 5ms/step - loss: 7.7668 - accuracy: 0.8787
Epoch 5/100
1229/1229 [==============================] - 5s 4ms/step - loss: 7.1732 - accuracy: 0.8826
Epoch 6/100
1229/1229 [==============================] - 5s 4ms/step - loss: 6.9139 - accuracy: 0.8866
Epoch 7/100
1229/1229 [==============================] - 5s 4ms/step - loss: 6.6420 - accuracy: 0.8902
Epoch 8/100
1229/1229 [==============================] - 4s 3ms/step - loss: 6.3833 - accuracy: 0.8932
Epoch 9/100
1229/1229 [==============================] - 4s 3ms/step - loss: 5.9333 - accuracy: 0.8969
Epoch 10/100
1229/1229 [==============================] - 5s 4ms/step -

1229/1229 [==============================] - 6s 5ms/step - loss: 3.3649 - accuracy: 0.9269
Epoch 80/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.5691 - accuracy: 0.9249
Epoch 81/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.2830 - accuracy: 0.9262
Epoch 82/100
1229/1229 [==============================] - 6s 5ms/step - loss: 3.3587 - accuracy: 0.9263
Epoch 83/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.4630 - accuracy: 0.9263
Epoch 84/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.4034 - accuracy: 0.9275
Epoch 85/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.4411 - accuracy: 0.9256
Epoch 86/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.3696 - accuracy: 0.9272
Epoch 87/100
1229/1229 [==============================] - 5s 4ms/step - loss: 3.1977 - accuracy: 0.9284
Epoch 88/100
1229/1229 [==============================] - 5s 4ms/step - loss:

In [34]:
encoder.categories_[0]

array(['Asphalt_Bump', 'Asphalt_Kurb', 'Asphalt_Manhole', 'Asphalt_Ride',
       'Cobblestone_Ride', 'Stop'], dtype=object)

In [35]:
y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix(y_test_class, y_pred_class)

273/273 [==============================] - 1s 3ms/step


array([[   0,    0,   14,   27,    5,    0],
       [   0,    0,    7,   42,   19,    0],
       [   0,    2,    7,  243,   43,    0],
       [   0,    0,    0, 5522,   96,   39],
       [   0,    0,    0,   33, 1028,    3],
       [   0,    0,    0,    6,    2, 1574]], dtype=int64)

In [36]:
nn_f1_s = f1_score(y_test_class, y_pred_class, average ="weighted")
nn_f1_s

0.9134353301831762

In [37]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        46
           1       0.00      0.00      0.00        68
           2       0.25      0.02      0.04       295
           3       0.94      0.98      0.96      5657
           4       0.86      0.97      0.91      1064
           5       0.97      0.99      0.98      1582

    accuracy                           0.93      8712
   macro avg       0.50      0.49      0.48      8712
weighted avg       0.90      0.93      0.91      8712



C:\Users\arockias\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arockias\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arockias\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
model.save(r"C:\Users\arockias\Desktop\SmartRadL\SmartRadL\Programs\Machine Learning\Models\NN_model")

INFO:tensorflow:Assets written to: C:\Users\arockias\Desktop\SmartRadL\SmartRadL\Programs\Machine Learning\Models\NN_model\assets
